# Similar to the code in "HydroShed_MeshGenerator.ipynb" to generate a mesh

In [ ]:
# Libraries

import numpy as np
from skimage.transform import resize
import pymartini
import rasterio
import pyvista as pv
from scipy.ndimage import map_coordinates
import settings
import utils

# matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
# for interactions
from mpl_toolkits.mplot3d import Axes3D

import TIN_engine
from TIN_engine import *
from TIN_draw import *
from TIN_drainage import *

# remove this (and possibly restart the kernel) if you don't want interactive plots
%matplotlib widget

# for reloading modules (specifically TIN_engine) during development
%load_ext autoreload
%autoreload 2

In [ ]:

dem_path = settings.WASHINGTON_SMALL
output_vtp = settings.DATA_DIR / "hyd_na_dem_30s_mesh_3d_corrected3.vtp"

# Load DEM and metadata
with rasterio.open(dem_path) as src:
    dem = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata_val = src.nodata

# Handle NoData values: mask and fill with NaN
dem_masked = dem.astype(np.float32)
if nodata_val is not None:
    dem_masked[dem_masked == nodata_val] = np.nan

# Optionally clip extreme elevation values but ignore NaNs
valid_vals = dem_masked[~np.isnan(dem_masked)]
lower_clip = np.percentile(valid_vals, 1)
upper_clip = np.percentile(valid_vals, 99)
dem_clipped = np.clip(dem_masked, lower_clip, upper_clip)

# --- Resize DEM safely

# we need a 2^k + 1 sized grid for the Martini algorithm to work
max_size = 1025
dem_size = dem_clipped.shape[0]
target_size = min(max_size, 2 ** int(np.floor(np.log2(dem_size - 1))) + 1)

# resize
dem_resized = resize(dem_clipped, (target_size, target_size), preserve_range=True, anti_aliasing=True)
dem_resized = np.ascontiguousarray(dem_resized.astype(np.float32))

# Build Martini mesh
grid_size = dem_resized.shape[0]
martini = pymartini.Martini(grid_size)
tile = martini.create_tile(dem_resized)

# Extract mesh at desired level
level = 10
vertices, triangles = tile.get_mesh(level)
vertices = np.array(vertices, dtype=np.float32).reshape(-1, 2) # put into 2 columns
triangles = np.array(triangles, dtype=np.int32).reshape(-1, 3) # put into 3 columns

rows = vertices[:, 0]
cols = vertices[:, 1]

# Map grid indices to real-world coordinates (x, y)
rows_int = np.clip(np.round(rows).astype(int), 0, dem_resized.shape[0] - 1)
cols_int = np.clip(np.round(cols).astype(int), 0, dem_resized.shape[1] - 1)
xs, ys = rasterio.transform.xy(transform, rows_int, cols_int)
xs = np.array(xs, dtype=np.float32)
ys = np.array(ys, dtype=np.float32)

# Bilinear interpolate elevation for fractional vertices (rows, cols)
coords = np.vstack([rows, cols])
zs = map_coordinates(dem_resized, coords, order=1, mode='nearest')

# Fix NaN values by replacing with nearby valid elevations (note: this does not seem to be needed)
nan_mask = np.isnan(zs)
if np.any(nan_mask):
    # Replace NaNs by nearest valid value (simple approach)
    zs[nan_mask] = np.nanmean(zs[~nan_mask])

# Normalize elevation (between 0 and 1) ignoring NaNs
zs_min = np.nanmin(zs)
zs_max = np.nanmax(zs)
zs = (zs - zs_min) / (zs_max - zs_min)

# Fix vertical exaggeration
z_scale = 0.01
zs_scaled = zs * z_scale

# Stack into final vertices (X, Y, Z)
vertices_3d = np.column_stack([xs, ys, zs_scaled])

# Optional: center horizontally for better visualization
#vertices_3d[:, 0] -= vertices_3d[:, 0].mean()
#vertices_3d[:, 1] -= vertices_3d[:, 1].mean()

mesh = get_mesh_from_triangles(triangles, vertices_3d)

In [ ]:
# Plot the mesh
pv.plot(mesh, jupyter_backend='static')

In [ ]:
# plot the verticies

# only plotting a subset here
num = 10000
X = xs[0:num] 
Y = ys[0:num]
Z = zs[0:num]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, s=0.5, c=Z, marker='o', cmap='viridis')

plt.show()

In [ ]:
# get highest point
max_index = np.nanargmax(zs)
highest_point = [xs[max_index], ys[max_index], zs[max_index]]
highest_point

In [ ]:
#triangles_subset = get_subset_of_triangles_from_bounds(triangles, [-116.5, 49.34, -116.52, 49.32], xs, ys)
radius = 0.015
bounds_around_highest = [highest_point[0] + radius, highest_point[1] + radius, highest_point[0] - radius, highest_point[1] - radius]

triangles_subset = get_subset_of_triangles_from_bounds(triangles, bounds_around_highest, xs, ys)

In [ ]:
triangle_objects, vertices = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)
print(len(triangle_objects), "triangle objects created.")
print(len(vertices), "vertex objects created.")

In [ ]:
lines = []

points = calculate_steepest_descent_line(highest_point, triangle_objects, triangles_subset, xs, ys, zs)
lines.append(points)

In [ ]:
has_flat_triangles(triangle_objects)
flat = get_flat_triangles(triangle_objects)
len(flat)
unflaten_triangles(triangle_objects)

In [ ]:
# ---- Drainage network calculation ----
drainage_outlet_nodes = create_drainage_network(triangle_objects)
print(len(drainage_outlet_nodes), "outlet nodes created.")

In [ ]:
# Visualize drainage network

# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

network_lines = []

def draw_node(node: Node, depth, color):
    if depth > 40:
        print("Max depth reached, stopping recursion.")
        return

    for upstream_node in node.upstream_nodes:
        draw_line_points(ax, node.point[0:2], upstream_node.point[0:2], color, linewidth=2)
        network_lines.append([node.point, upstream_node.point])
        draw_node(upstream_node, depth + 1, color)

for outlet in drainage_outlet_nodes:
    draw_point(ax, outlet.point[0:2], markersize=4)
    random_color = np.random.rand(3,)
    draw_node(outlet, 0, random_color)

print("Done drawing network.")

plt.show()

In [ ]:
def show_lines(plotter: pv.Plotter):
    # draw lines
    z_offset = 0.0001
    previous_percentange = 0

    i = 0
    for line in network_lines:
        prev_point = line[0]

        new_line = []
        p2 = None
        for point in line[1:]:
            p1 = [prev_point[0], prev_point[1], (prev_point[2] * z_scale) + z_offset]
            p2 = [point[0], point[1], (point[2] * z_scale) + z_offset]
            new_line.append(p1)
            new_line.append(p2)
            prev_point = point

        plotter.add_lines(np.array(new_line), color='blue', width=5)

        i += 1
        current_percentage = int((i / len(network_lines)) * 100)
        if current_percentage != previous_percentange:
            print(f"Progress: {current_percentage}%")
            previous_percentange = current_percentage

    print("Done drawing lines.")
    

p = pv.Plotter()
p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')
show_lines(p)
p.show() # fully interactive

In [ ]:
channel_lines = calculate_channel_flow_lines(vertices, triangle_objects)
print(len(channel_lines))
print(channel_lines)

In [ ]:
max_length = 0
for line in channel_lines:
    if len(line) > max_length:
        max_length = len(line)
max_length

In [ ]:
# get vertex closest to coord
coord = (-116.56625, 49.50462)
closest_vertex = None
min_dist = float('inf')
for vertex in vertices.values():
    dist = np.sqrt((vertex.x - coord[0])**2 + (vertex.y - coord[1])**2)
    if dist < min_dist:
        min_dist = dist
        closest_vertex = vertex

print(closest_vertex.x, closest_vertex.y, closest_vertex.z)

In [ ]:
print(closest_vertex)

In [ ]:
# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

# draw lines
for line in channel_lines:
    thickness = 1
    thickness_growth = 1
    color = (0, 0, 1)
    color_growth = (0.05, 0, 0)

    prev_point = line[0]
    #draw_vertex(ax, prev_point, markersize=5)
    for point in line[1:]:
        draw_line(ax, [prev_point, point], "blue", linewidth=thickness)
        color = (min(color[0] + color_growth[0], 1), min(color[1] + color_growth[1], 1), min(color[2] + color_growth[2], 1))
        thickness += thickness_growth
        prev_point = point

draw_vertex(ax, closest_vertex, markersize=8)

"""
verts = get_vertices_on_area_bounds(vertices)
verts.extend(get_vertices_in_pit(vertices))
for vert in verts:
    draw_vertex(ax, vert, markersize=5)

    ordered = get_all_triangles_and_edges_at_point(vert, triangle_objects)

    previous_item = None
    next_item = None
    j = 0
    next_point = None
    for item in ordered:
        previous_item = ordered[(j - 1) % len(ordered)]
        next_item = ordered[(j + 1) % len(ordered)]

        if isinstance(item, Triangle):
            pass # do nothing for triangles
        else: # must be an edge
            draw_line(ax, [item[0], item[1]], "green", linewidth=2)

            if item[0].id != vert.id:
                item = (item[1], item[0]) # make first vertex be the given vertex
            
            #lies_in_channel = edge_lies_in_channel(item[0], item[1], previous_item, next_item)
            lies_in_channel = False
            ascending = item[0].z <= item[1].z

            # next_item and previous_item should be triangles adjacent to this edge
            if ascending and lies_in_channel:
                next_point = get_other_vertex_from_edge(vert, item)
                next_lines = channel_flow_line_helper(next_point, triangles) # keep going up the channel
                new_lines = [[item[1]]]
                for line in next_lines:
                    new_line = [item[1]]
                    new_line.extend(line)
                    new_lines.append(new_line)
                
                lines.extend(new_lines)"""

print("Done drawing lines.")

plt.show()

In [ ]:
# steepest descent lines
lines = []

#points = calculate_steepest_descent_line(highest_point, triangle_objects, triangles_subset, xs, ys, zs)
#lines.append(points)

# claculate descent lines
max_tries = float('inf')
for triangle in triangle_objects: 
    start_point = triangle.get_centroid().coord()
    points = calculate_steepest_descent_line(start_point, triangle_objects, triangles_subset, xs, ys, zs)
    #if len(points) > 4:
    lines.append(points)
    
    max_tries -= 1
    if max_tries <= 0:
        break

In [ ]:
# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

# draw lines
for line in lines:
    thickness = 1
    thickness_growth = 0.1
    color = (0, 0, 1)
    color_growth = (0.05, 0, 0)

    prev_point = line[0]
    for point in line[1:]:
        draw_line_points(ax, prev_point, point, "blue", linewidth=1)
        color = (min(color[0] + color_growth[0], 1), min(color[1] + color_growth[1], 1), min(color[2] + color_growth[2], 1))
        thickness += thickness_growth
        prev_point = point

print("Done drawing lines.")

plt.show()

In [ ]:
p = pv.Plotter()
p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')

z_offset = 0.0001
previous_percentange = 0

i = 0
for line in channel_lines:
    prev_point = line[0]

    new_line = []
    p2 = None
    for point in line[1:]:
        p1 = [prev_point.x, prev_point.y, (prev_point.z * z_scale) + z_offset]
        p2 = [point.x, point.y, (point.z * z_scale) + z_offset]
        new_line.append(p1)
        new_line.append(p2)
        prev_point = point

    p.add_lines(np.array(new_line), color='blue', width=5)

    i += 1
    current_percentage = int((i / len(channel_lines)) * 100)
    if current_percentage != previous_percentange:
        print(f"Progress: {current_percentage}%")
        previous_percentange = current_percentage

print("Done drawing lines.")

#p.show(jupyter_backend='static')
#p.show(jupyter_backend='html') # simple interactive
p.show() # fully interactive

In [ ]:
p = pv.Plotter()
p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')

z_offset = 0.0005
previous_percentange = 0

i = 0
for line in lines:
    prev_point = line[0]

    new_line = []
    p2 = None
    for point in line[1:]:
        p1 = [prev_point[0], prev_point[1], (prev_point[2] * z_scale) + z_offset]
        p2 = [point[0], point[1], (point[2] * z_scale) + z_offset]
        new_line.append(p1)
        new_line.append(p2)
        prev_point = point

    #p.add_lines(np.array(new_line), color='blue', width=5)

    i += 1
    current_percentage = int((i / len(lines)) * 100)
    if current_percentage != previous_percentange:
        print(f"Progress: {current_percentage}%")
        previous_percentange = current_percentage

print("Done drawing lines.")

#p.show(jupyter_backend='static')
#p.show(jupyter_backend='html') # simple interactive
p.show() # fully interactive

In [ ]:
mask = (xs < -116.5) & (xs > -116.52) & (ys < 49.34) & (ys > 49.32)
#mask = (xs < -116.505) & (xs > -116.510) & (ys > 49.3275) & (ys < 49.3325)

verticies_subset = np.where(mask)
m = np.isin(triangles, verticies_subset)
triangles_subset = triangles[np.all(m, axis=1)]

In [ ]:
# testing
test = np.array([[1, 2, 3], [4, 5, 6], [1, 2, 6], [1, 2, 3], [1, 2, 3]])
vert = np.array([1, 2, 3])

m = np.isin(test, vert)

test[np.all(m, axis=1)] # all the verticies have to match for each triangle

In [ ]:
rng = np.random.default_rng()

In [ ]:
#start_point = [-116.5072, 49.33]
start_point = [-116.5072, 49.3308]
#start_point = [-116.5077, 49.3285]

In [ ]:
# Draws a nice graph that shows the lines of steepest descent from each triangle's centroid (see similar in Johnes. et al., p. 1241)

# plot some vertices in 2D
X = xs[mask]
Y = ys[mask]
Z = zs[mask]

fig = plt.figure()
ax = fig.add_subplot(111)

POINT_MARKER_SIZE = 1

# TODO: make all functions use triangle objects
#triangle_objects = convert_to_triangle_objects(triangles_subset, xs, ys, zs)
triangle_objects, vers = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)

# draw triangles
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

for a_triangle in triangle_objects:
    thickness = 1
    thickness_growth = 0.1
    start_point = a_triangle.get_centroid().coord()[0:2]

    #draw_point(ax, start_point, POINT_MARKER_SIZE)

    tri = get_triangle_at(start_point, triangles_subset, xs, ys, zs)
    #draw_triangle(ax, tri, "#FF0000FF", xs, ys)

    s_point = start_point
    for i in range(1, 100):
        full_tri = get_full_3D_triangle(tri, xs, ys, zs)

        descent = calculate_steepest_descent(full_tri)
        descent = descent / np.linalg.norm(descent)
        descent *= 0.001
        #ax.plot([s_point[0], s_point[0] + descent[0]], [s_point[1], s_point[1] + descent[1]], "-", color="red", linewidth=1)

        next_point, adj_tri, v1, v2 = get_point_and_adj_triangle_from_descent(tri, s_point, descent, xs, ys, zs, triangles_subset)
        if next_point is None:
            print("next_point should not be None, stopping at iteration ", i)
            break

        #ax.plot(next_point[0], next_point[1], 'o', markersize=POINT_MARKER_SIZE)
        draw_line_points(ax, s_point, next_point, "blue", linewidth=thickness)
        thickness += thickness_growth
        if adj_tri is None:
            print("No adjacent triangle found, stopping at iteration ", i)
            break

        #draw_triangle(ax, adj_tri, "#00FF00FF", xs, ys)

        full_adj_tri = get_full_3D_triangle(adj_tri, xs, ys, zs)
        descent_adj = calculate_steepest_descent(full_adj_tri) # gh from p. 1239, Jones et al.

        full_adj_tri = make_triangle_counterclockwise(full_adj_tri)

        coord1 = get_real_vertex_3D(v1, xs, ys, zs)
        coord2 = get_real_vertex_3D(v2, xs, ys, zs)
        v1 = Vertex(coord1[0], coord1[1], coord1[2], v1)
        v2 = Vertex(coord2[0], coord2[1], coord2[2], v2)

        if (find_row_index(full_adj_tri, v1.coord()) + 1) % 3 == find_row_index(full_adj_tri, v2.coord()): # if v1 comes before v2 
            ij = v2.coord() - v1.coord()
        else: # v2 comes before v1
            # swap
            temp = v1
            v1 = v2
            v2 = temp

            ij = v2.coord() - v1.coord()

        # direction of the adj. triangle: if positive then adj. triangle slopes toward the current edge
        direction = utils.cross_2D(np.array(descent_adj), ij[0:2])
        #print(direction)
        #print(i)

        #print(full_tri)
        #print(full_adj_tri)
        
        if direction < 0:
            tri = adj_tri
            s_point = next_point
            continue
        else:
            lowest = None
            if v1.z > v2.z:
                lowest = v2
            else:
                lowest = v1
            
            draw_line_points(ax, next_point, lowest.coord()[0:2], "blue", linewidth=thickness)
            thickness += thickness_growth


            stop = False
            while True:
                #draw_vertex(ax, lowest, POINT_MARKER_SIZE)

                #triangles_at_point = get_all_triangles_with_point(lowest, triangle_objects)
                #edges_at_point = get_all_edges_with_point(lowest, triangles_at_point)
                #for t in triangles_at_point:
                #    draw_triangle_object(ax, t, "blue")
                
                #for e in edges_at_point:
                #    draw_line(ax, e, "green")
                
                # TODO: do not check the triangles/edges that have already been visited

                ordered = get_all_triangles_and_edges_at_point(lowest, triangle_objects)
                color = (1, 0, 0)
                previous_item = None
                next_item = None
                j = 0
                next_point = None
                next_triangle = None
                for item in ordered:
                    previous_item = ordered[(j - 1) % len(ordered)]
                    next_item = ordered[(j + 1) % len(ordered)]

                    if isinstance(item, Triangle):
                        #draw_triangle_object(ax, item, color)
                        if test_triangle(lowest, item):
                            #draw_triangle_object(ax, item, "green")
                            next_triangle = item
                    else: # must be an edge
                        #draw_line(ax, item, color)
                        # next_item and previous_item should be triangles adjacent to this edge
                        if test_edge(lowest, item, next_item, previous_item):
                            draw_line(ax, item, "blue", linewidth=thickness)
                            thickness += thickness_growth
                            next_point = get_other_vertex_from_edge(lowest, item)
                    
                    color = ((color[0] - (1.0 / 30.0)) % 1.0, color[1], color[2])
                    j += 1
                
                if next_point is not None:
                    lowest = next_point
                    #print("Continuing from another next_vertex")
                    continue
                elif next_triangle is not None:
                    tri = next_triangle.convert_to_indices()
                    s_point = lowest.coord()[0:2]
                    #print("Continuing from a triangle")
                    break # and then continue the outer for loop
                else:
                    # need to stop here
                    stop = True
                    #print("Stopping here")
                    break

            
            if stop:
                #print("Reached lowest point at iteration ", i)
                break
            else:
                continue

#ax.scatter(X, Y, s=10, c=Z, marker='o', cmap='viridis')

plt.show()